In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def accuracy(preds, labels):
    """Calculates accuracy given two numpy arrays"""
    
    return np.mean(preds == labels) * 100

def confusion_matrix(preds, labels):
    """Creates a 10 x 10 confusion matrix given two numpy arrays"""
    
    matrix = np.zeros((10,10))
    for i in range(len(preds)):
        matrix[preds[i],labels[i]] += 1
    return matrix    

def precision_and_recall(preds, labels):
    """Returns individual Precision and Recall values of each class"""
    
    matrix = confusion_matrix(preds, labels)
    r = []
    p = []
        
    for i in range(10):
        TP = float(matrix[i,i])
        TP_FP = np.sum(matrix[i,:])
        TP_FN = np.sum(matrix[:,i])
        
        recall = "NA" if TP_FN == 0 else TP / TP_FN
        precision = "NA" if TP_FP == 0 else TP / TP_FP
        r.append(recall)
        p.append(precision)
    
    return (p, r)


def fscore(preds, labels):
    """Calculates macro f score given two numpy arrays"""
    
    p, r = precision_and_recall(preds, labels)
    
    precision = 0
    recall = 0
    
    count = 0.0
    for x in p:
        if x != "NA":
            count += 1
            precision += x
    precision /= count
    
    count = 0.0
    for x in r:
        if x != "NA":
            count += 1
            recall += x
    recall /= count
    
    return 2*precision*recall/(precision+recall)*100


In [3]:
def read_csv_into_matrix(path):
    df = []
    colnames = None
    with open(path, "r") as f:
        colnames = {i:name[1:-1] for i, name in enumerate(f.readline().strip().split(';'))}
        line = f.readline()
        while line:
            df.append([float(x) for x in line.strip().split(";")])
            line = f.readline()
    return colnames, np.asarray(df, dtype=float)
    

In [4]:
""" 
Format of data (Numpy matrix)
    - Each row is one sample
    - Each column is a particular feature
    - Last column is the label
Header is the column names of all the features
"""
header, data = read_csv_into_matrix("winequality-white.csv")

In [5]:
def unique_vals(df, col):
    """Find the unique values for a column in a dataset."""
    return np.unique(df[:,col])

In [6]:
def class_counts(df):
    """Counts the number of each type of example in a dataset."""
    return np.unique(df[:,-1], return_counts=True)

In [7]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [8]:
class Question:
    """A Question is used to partition a dataset."""

    def __init__(self, column, value):
        """Stores column number and value of the question"""
        
        self.column = column
        self.value = value
        
    def match(self, row):
        """Answers question for a particular row"""
        
        if is_numeric(self.value):
            return row[self.column] >= self.value
        else:
            return row[self.column] == self.value

    def partition(self, df):
        """Splits given df based on question"""
        
        true_df = None
        false_df = None
        index = None
        if is_numeric(self.value):
            index = df[:,self.column] >= self.value
        else:
            index = df[:self.column] == self.value
        true_df = df[index]
        false_df = df[~index]
        return true_df, false_df

    def __repr__(self):
        """Helper method to print question in readable form"""
        
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is {} {} {} ?".format(header[self.column], condition, str(self.value))

In [9]:
# Testing on a toy dataset
toy = np.random.randn(50, 10) * 100
toy.shape

(50, 10)

In [10]:
q = Question(1, 0)
t, f = q.partition(toy)

In [11]:
print(toy.shape)
print(t.shape)
print(f.shape)

(50, 10)
(25, 10)
(25, 10)


In [12]:
def entropy(df):
    """Calculate entropy of data with respect to the last column (labels)"""
    
    labels, counts = class_counts(df)
    probs = counts / float(df.shape[0])
    entropy = - np.sum(probs * np.log2(probs))
    return entropy

In [13]:
entropy(data)

1.8617149332355558

In [14]:
def info_gain(left, right, current_entropy):
    """Information Gain.

    The Entropy of the starting node, minus the weighted entropy of
    two child nodes.
    """
    p = float(left.shape[0]) / (left.shape[0] + right.shape[0])
    return current_entropy - p * entropy(left) - (1 - p) * entropy(right)

In [15]:
info_gain(t, f, entropy(toy))

1.0000000000000009

In [16]:
def find_best_split(df):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain.
    Is surprisingly fast.
    If slow:
        - Try 1000 random values in a particular column instead of all values
        - Dynamic Programming approach to find best split for each feature
    """
    
    best_gain = 0  
    best_question = None 
    
    current_entropy = entropy(df)
    
    # Loop over each feature and its value to find best info_gain
    n_features = df.shape[1] - 1  
    for col in range(n_features): 
        values = np.unique(df[:,col])  
        for val in values:  

            question = Question(col, val)

            # Try splitting the dataset
            true_rows, false_rows = question.partition(df)

            # Skip this split if it doesn't divide the dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_entropy)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [17]:
print(find_best_split(data))

(0.1433405667634391, Is alcohol >= 10.9 ?)


In [18]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [19]:
class Leaf:
    """A Leaf node classifies data.
        
    predictions is a dictionary of counts of classes in this leaf.
    Output is usually the majority class.
    """

    def __init__(self, rows):
        counts = np.column_stack(class_counts(rows))
        self.predictions = {row[0]:row[1] for row in counts}

In [20]:
class Decision_Tree:
    
    def __init__(self, max_depth=32, min_sample_split=2):
        self.depth = max_depth
        self.min_sample_split = min_sample_split
        
        
    def train(self, df):
        """Builds tree based on dataset"""
        self.root = self._build_tree(df, self.depth)
        
        
    def _build_tree(self, df, depth):
        """Builds the tree recursively"""

        # If number of samples is less than required, we cannot split.
        if df.shape[0] < self.min_sample_split:
            return Leaf(df)
        
        # Pick best question by maximizing information gain
        gain, question = find_best_split(df)

        # If there is not gain, or we have reached max depth
        if gain == 0 or depth == 0:
            return Leaf(df)
        

        # Partition and recursively build left and right branches
        true_rows, false_rows = question.partition(df)
        true_branch = self._build_tree(true_rows, depth-1)
        false_branch = self._build_tree(false_rows, depth-1)

        return Decision_Node(question, true_branch, false_branch)
    
    def predict(self, row):
        """Returns prediction for single row"""
        
        prediction = self._classify(row, self.root)
        return prediction
    
    def _classify(self, row, node):
        """Recursively goes down the tree to return prediction"""

        # Base case: At a leaf node
        if isinstance(node, Leaf):
            maxcount = 0
            maxlabel = None
            for k, v in node.predictions.items():
                if v >= maxcount:
                    maxcount = v
                    maxlabel = k
            return maxlabel

        # If not, match with question.
        if node.question.match(row):
            return self._classify(row, node.true_branch)
        else:
            return self._classify(row, node.false_branch) 
        
    def test(self, df, labels):
        """Returns accuracy and fscore of test dataset"""
        # Not vectorized.
        
        preds = np.zeros(df.shape[0])
        for i, row in enumerate(df):
            preds[i] = self.predict(row)
            
        a = accuracy(preds.astype(int), labels.astype(int))
        f = fscore(preds.astype(int), labels.astype(int))
        
        return a, f

In [21]:
# Hyperparameters: Max depth, Min number of splits.
model = Decision_Tree(32, 5)

In [22]:
# Train takes data and labels together
model.train(data)

In [27]:
# Test takes data and labels separately.
model.test(data[:,:-1], data[:,-1])

(95.48795426704777, 0.5876952659660883)

In [21]:
def create_folds(data, num_folds):
    """Splits the given data into num_folds folds"""
    # Initial shuffle
    np.random.shuffle(data)
    # Number of points in each fold
    cut = int(math.ceil(len(data)/float(num_folds)))

    # Divide the original data into folds and save each separately.
    folds = []
    for i in range(0, len(data), cut):
        folds.append(data[i:i+cut,:])
    return folds


# 4 Fold cross validation
num_folds = 4
folds = create_folds(data, num_folds)

# Ratio of train_data to use as validation
ratio = 0.2

    
result_f = []
result_a = []
for minsample in [3]:
    for depth in range(18,27,2):  
        print ""
        print "Hyper-parameters:"
        print "Depth: {}".format(depth)
        print "Min-samples split: {}".format(minsample)

        train_a_list = []
        train_f_list = []
        valid_f_list = []
        valid_a_list = []
        test_a_list = []
        test_f_list = []

        #Loop over all folds using different test sets each time
        for i in range(num_folds):

            test_data = folds[i]
            # Anything other than the test fold is used as training
            train_folds = [x for x in range(num_folds) if x != i]
            rem_data = folds[train_folds[0]]
            for x in train_folds[1:]:
                rem_data = np.concatenate((rem_data, folds[x]))

            # Split train and validation depending on ratio.
            split = int(len(rem_data)*(1-ratio))
            train_data = rem_data[:split,:]
            validation_data = rem_data[split:,:]

            # Training
            model = Decision_Tree(depth, minsample)
            model.train(train_data)

            # Validation and testing
            train_acc, train_f = model.test(train_data[:,:-1], train_data[:,-1])
            valid_acc, valid_f = model.test(validation_data[:,:-1], validation_data[:,-1])        
            test_acc, test_f = model.test(test_data[:,:-1], test_data[:,-1])


            print "\nFold-{}:".format(i+1)
            print "Train: F1 Score: {:.1f}, Accuracy: {:.1f}".format(train_f, train_acc)
            print "Validation: F1 Score: {:.1f}, Accuracy: {:.1f}".format(valid_f, valid_acc)
            print "Test: F1 Score: {:.1f}, Accuracy: {:.1f}".format(test_f, test_acc)

            train_a_list.append(train_acc)
            train_f_list.append(train_f)
            valid_f_list.append(valid_f)
            valid_a_list.append(valid_acc)
            test_a_list.append(test_acc)
            test_f_list.append(test_f)


        # Averaging
        avg_train_acc = sum(train_a_list) / 4
        avg_train_f = sum(train_f_list) / 4
        avg_v_acc = sum(valid_a_list) / 4
        avg_v_f = sum(valid_f_list) / 4
        avg_t_acc = sum(test_a_list) / 4
        avg_t_f = sum(test_f_list) / 4

        print "\nAverage:"
        print "Train: F1 Score: {:.1f}, Accuracy: {:.1f}".format(avg_train_f, avg_train_acc)
        print "Validation: F1 Score: {:.1f}, Accuracy: {:.1f}".format(avg_v_f, avg_v_acc)
        print "Test: F1 Score: {:.1f}, Accuracy: {:.1f}".format(avg_t_f, avg_t_acc)

        result_f.append(avg_v_f)
        result_a.append(avg_v_acc)


Hyper-parameters:
Depth: 18
Min-samples split: 3

Fold-1:
Train: F1 Score: 91.0, Accuracy: 97.1
Validation: F1 Score: 33.0, Accuracy: 55.1
Test: F1 Score: 33.3, Accuracy: 56.7

Fold-2:
Train: F1 Score: 93.9, Accuracy: 98.0
Validation: F1 Score: 32.7, Accuracy: 53.7
Test: F1 Score: 39.2, Accuracy: 57.1

Fold-3:
Train: F1 Score: 92.9, Accuracy: 97.6
Validation: F1 Score: 31.7, Accuracy: 53.5
Test: F1 Score: 36.8, Accuracy: 58.9

Fold-4:
Train: F1 Score: 90.1, Accuracy: 97.3
Validation: F1 Score: 36.7, Accuracy: 55.9
Test: F1 Score: 36.7, Accuracy: 57.2

Average:
Train: F1 Score: 92.0, Accuracy: 97.5
Validation: F1 Score: 33.5, Accuracy: 54.6
Test: F1 Score: 36.5, Accuracy: 57.5

Hyper-parameters:
Depth: 20
Min-samples split: 3

Fold-1:
Train: F1 Score: 91.8, Accuracy: 97.8
Validation: F1 Score: 32.7, Accuracy: 54.7
Test: F1 Score: 33.0, Accuracy: 56.5

Fold-2:
Train: F1 Score: 94.1, Accuracy: 98.5
Validation: F1 Score: 32.6, Accuracy: 53.6
Test: F1 Score: 39.2, Accuracy: 57.1

Fold-3:
T